## Libraries

In [8]:
import win32com.client as win32
import os
import time

from config import EMAIL, SUBJECT, API_KEY_WAPI, PROXI_USER, PROXI_PWD


from requests import Request, Session
from requests.exceptions import ConnectionError, Timeout, TooManyRedirects
import json

import pandas as pd

import pandas as pd
import requests
from requests_ntlm import HttpNtlmAuth

from bs4 import BeautifulSoup
from tqdm import tqdm

from datetime import datetime


### Traducciones condicion clima

In [9]:
weather_translate = {
    'Sunny' : 'Soleado',
    'Rain' : 'Lluvia',
    'Cloudy' : 'Nublado',
    'Parthly Cloudy' : 'Parcialmente Nublado',
    'Wind' : 'Viento',
    'Storm' : 'Tormenta',
    'Snow' : 'Nieve'
}

## Data Collection

In [10]:
query = 'Juarez'
api_key = API_KEY_WAPI


url = "http://api.weatherapi.com/v1/forecast.json?key="+api_key+"&q="+query+"&days=1&aqi=no&alerts=no"
#url = 'http://api.weatherapi.com/v1/forecast.json?key=6284cbda493d471fbef03655242409&q=ciudad juarez&days=1&aqi=no&alerts=no'


In [11]:
proxies = {
    'http': 'http://rb-proxy-na.bosch.com:8080',
    'https': 'http://rb-proxy-na.bosch.com:8080'
}

auth = HttpNtlmAuth(PROXI_USER, PROXI_PWD)

response = requests.get(url, proxies=proxies, auth=auth)

if response.status_code == 200:
    data = response.json()
    print("Consulta realizada exitosamente")
else:
  print(f"Error: {response.status_code}, {response.text}")

Consulta realizada exitosamente


### Data Understanding

In [12]:
data.keys()

dict_keys(['location', 'current', 'forecast'])

In [13]:
data['forecast']['forecastday'][0].keys()

dict_keys(['date', 'date_epoch', 'day', 'astro', 'hour'])

In [14]:
len(data['forecast']['forecastday'][0]['hour'])

24

In [15]:
data['forecast']['forecastday'][0]['hour'][1]['time'].split()[0] #fecha

'2024-09-30'

In [16]:
int(data['forecast']['forecastday'][0]['hour'][1]['time'].split()[1].split(':')[0]) #hora

1

In [17]:
data['forecast']['forecastday'][0]['hour'][0]['condition']['text'] #condition

'Clear '

In [18]:
data['forecast']['forecastday'][0]['hour'][0]['temp_c'] #temperatura


24.7

In [19]:
data['forecast']['forecastday'][0]['hour'][0]['will_it_rain'] #llovera (1 si, 0 no      )

0

In [20]:
data['forecast']['forecastday'][0]['hour'][0]['chance_of_rain'] #Posbilidad de lluvia


0

In [21]:
data['forecast']['forecastday'][0]['day']['maxtemp_c'] #maxtemp

34.1

In [22]:
data['forecast']['forecastday'][0]['day']['daily_chance_of_rain'] #%lluvia

0

In [23]:
data['forecast']['forecastday'][0]['day']['condition']['text'] #condicion clima

'Sunny'

In [24]:
data['forecast']['forecastday'][0]['date'] #fecha

'2024-09-30'

### Data Transformation - Version original proyecto

In [25]:
def get_forecast(data, i):
    fecha = data['forecast']['forecastday'][0]['hour'][i]['time'].split()[0] #fecha
    hora = int(data['forecast']['forecastday'][0]['hour'][i]['time'].split()[1].split(':')[0]) #hora
    condicion = data['forecast']['forecastday'][0]['hour'][i]['condition']['text'] #condition
    temp = data['forecast']['forecastday'][0]['hour'][i]['temp_c'] #temperatura
    rain = data['forecast']['forecastday'][0]['hour'][i]['will_it_rain'] #llovera (1 si, 0 no      )
    prob_rain = data['forecast']['forecastday'][0]['hour'][i]['chance_of_rain'] #Posbilidad de lluvia

    return fecha, hora, condicion, temp, rain, prob_rain

In [26]:
datos = []

for i in tqdm(range(len(data['forecast']['forecastday'][0]['hour'])), colour = 'green'):
    datos.append(get_forecast(data, i))

100%|██████████| 24/24 [00:00<?, ?it/s]


In [27]:
col = ['fecha', 'hora', 'condicion', 'temp', 'rain', 'prob_rain']
df_weather = pd.DataFrame(datos, columns=col)
df_weather


,fecha,hora,condicion,temp,rain,prob_rain
0,2024-09-30,0,Clear,24.7,0,0
1,2024-09-30,1,Clear,24.0,0,0
2,2024-09-30,2,Clear,23.3,0,0
3,2024-09-30,3,Clear,22.5,0,0
4,2024-09-30,4,Clear,21.8,0,0
5,2024-09-30,5,Clear,21.2,0,0
6,2024-09-30,6,Clear,20.7,0,0
7,2024-09-30,7,Clear,20.4,0,0
8,2024-09-30,8,Sunny,21.5,0,0
9,2024-09-30,9,Sunny,23.9,0,0


In [28]:
df_rain = df_weather[(df_weather['rain'] == 1) & (df_weather['hora']> 6) & (df_weather['hora'] < 22)]
df_rain = df_rain[['hora', 'condicion']]
df_rain.set_index('hora', inplace = True)

In [29]:
df_rain

,condicion
hora,


### Data Transformation - Version Personal

In [30]:
date = data['forecast']['forecastday'][0]['date'] #fecha
maxTemp= data['forecast']['forecastday'][0]['day']['maxtemp_c'] #maxtemp
minTemp= data['forecast']['forecastday'][0]['day']['mintemp_c'] #mintemp
chanceRain = data['forecast']['forecastday'][0]['hour'][0]['chance_of_rain'] #Posbilidad de lluvia
weatherCondition = data['forecast']['forecastday'][0]['day']['condition']['text'] #condicion clima

weatherConditionTranslate = weather_translate[weatherCondition]

if not weatherConditionTranslate:
    weatherConditionTranslate = weatherCondition

In [31]:
mensaje = 'Hola, este es el pronostico del clima del dia de hoy: ' + date + '\nTemperadura Maxima: '+ str(maxTemp) +' °C\nTemperadura Minima: '+ str(minTemp) +' °C\nPosibilidad de Lluvia: '+ str(chanceRain) + '%' +'\nCondicion: '+ str(weatherConditionTranslate)

In [32]:
print(mensaje)

Hola, este es el pronostico del clima del dia de hoy: 2024-09-30
Temperadura Maxima: 34.1 °C
Temperadura Minima: 20.1 °C
Posibilidad de Lluvia: 0%
Condicion: Soleado


In [33]:
outlook = win32.Dispatch('outlook.application')
mail = outlook.CreateItem(0)
mail.TO = EMAIL
mail.Subject = SUBJECT
mail.Body = mensaje

In [34]:
mail.Send()